## Movie pipeline


**Main idea**: define UV map for first frame. Then shrink-wrap the first frame onto the mesh of the next frame (i.e. set vertex positions to the closest point on next frame's mesh, with an optional step of affine registration in between). On the shrink-wrapped mesh, we have our UV map, so we can use it for pullbacks. Best to carry out a little (corrective/Taubin) smoothing of shrink-wrap target and wrapped mesh to ensure better results.

**Main problem**: However, shrink-wrap runs into problems when large deformations appear which make remeshing necessary. 

**Possible solutions**:

- Remeshing. In blender, remeshing does _not_ preserve UV info (since you create a new mesh).
    - Pymeshlab: what is the behavior of `meshing_isotropic_explicit_remeshing` ? 

- Instead of deforming mesh of first frame, copy the UV info onto the target mesh. Possible in pymeshlab and blender. Problem: this will create issues at seams - the UV info is interpolated to copy, and this will create bad results where the UV map is discontinouous. Option - only do this from time to time?
    - Pymeshlab: `transfer_attributes_per_vertex`
    
- Subdivision. This creates a new, finer topology, which can accomodate the shape transformations. But the new faces are a superset of the old faces, and hence the UV info is preserved. Applying subdivision _everywhere_ does not help, since this greatly increases the size of the mesh each time. We only want to subdivide where it is necessary.  Note: Catmull-Clark is not great on triangular meshes, made for quads. Multiresolution Modifier in blender? Does not seem to subdivide only where necessary. Adaptive resolution in blender? Loop subdivision algorithm? What about libigl? 
    - Pymeshlab: ` meshing_surface_subdivision_loop`/`meshing_surface_subdivision_ls3_loop`/`meshing_surface_subdivision_butterfly`/`meshing_surface_subdivision_midpoint`. Can select which faces/edges to refine! Very helpful potentially is `meshing_vertex_attribute_seam`.
    - Libigl: `loop`/`loop_subdivision_matrix`. This is _uniform_ remeshing.
    - Trimesh: `trimesh.remesh.subdivide_to_size`. Unfortunately returns a "triangle soup" not an actual mesh.

**Other improvements which may help** 

- What happens if you start with a very deformed mesh and go the other way, i.e. apply your UV map in reverse? In general it is better to go from deformed to less deformed. In particular for growing surfaces, reverse in time is easiest way to go for sure.

- For better quality, create an average shape for making your initial UV map? Like a prototype mesh. Could do so by shrink-wrapping a mesh onto a few targets, and then averaging vertex positions, plus smooth and isotropic remeshing.

**Outlier filtering**

For mesh project.

**Other notes**

- Blender make multi-color surfaces (using texture and material preview) and Semi-Transparent too. Can surely be done without UV map (texture coordinates I think)

**Other options**

The other option, generally speaking, is to UV map from each timepoint mesh separately. Then use a reproducible algorithm of sorts to make sure one has consistent maps. Overall this is the strategy of Noah and Dillon. But this interferes with the idea of a graphical, interactive way of making UV maps! Maybe we can still learn something from the tubular paper.